In [2]:
%matplotlib inline
import matplotlib as mlp
import scipy
import numpy as np
import sympy
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import scale
from scipy.stats.stats import spearmanr 
from sklearn.linear_model import LinearRegression
from sklearn import cross_validation
from sklearn import metrics
import scipy.sparse as sps
import math
from bs4 import BeautifulSoup
import math as mt
import requests
import re
import fileinput
import operator
# import sys


In [3]:
#TODO :
# 1. Create columns for genres
# 2. based upon correlation with score vs genres, we can make our final predict_score better

df_book_info = pd.read_csv('goodreads_data_test.csv', sep=',')

#### Remove the unnecessary row and reset index
df_book_info = df_book_info.drop(df_book_info.index[[1]])
df_book_info = df_book_info.reset_index(drop=True)

#### Make a new column for award count
df_book_info['Awards List']=df_book_info['Awards List'].fillna('')
count = df_book_info['Awards List'].map(lambda x: len(str(x).split(',')) if x!='' else 0)
df_book_info['Award Count'] = count

#### Drop not required unnamed column
df_book_info = df_book_info.drop(df_book_info.columns[[0, 1]], axis=1)

#### Clean book name for merging
book_name_list =  df_book_info['Book Name'].tolist()
new_list = []
for one_string in book_name_list:
    m = re.search("(.*?)(\(|$)", one_string)
    new_list.append(m.group(1).strip())

#### Set new book name
df_book_info['New Book Name'] = new_list

#### Drop duplicates
df_book_info.drop_duplicates('New Book Name',inplace=True)
df_book_info = df_book_info.reset_index(drop=True)

#### Clean Number of reviews
df_book_info['Num_reviews'] = df_book_info['Num_reviews'].astype('str').map(lambda x: int(str(x).replace(',', '')) if (x!='nan') else np.NaN)

#### Calculate number of Characters
df_book_info['Num Characters'] = df_book_info['Characters'].astype('str').map(lambda x: 0 if (x=='nan' or x=='[]') else len(x.split(',')))

#### Clean stars
df_book_info['Stars'] = pd.to_numeric(df_book_info['Stars'])
print "Is any star nan: ",df_book_info['Stars'].map(lambda x: mt.isnan(x)).any(0)

#### Clean Other editions
print "Number of non digit editions: ",df_book_info['Other editions'].map(lambda x: not x.isdigit()).sum()
df_book_info['Other editions'] = df_book_info['Other editions'].map(lambda x: int(x) if x.isdigit() else 0)

#### Split star counts
temp = df_book_info['Star votes'].astype('str').map(lambda x: x[1:-1].split(',') if (x!='nan') else '')
for star_rating in range(5,0,-1):
    print star_rating
    df_book_info['Star_count_'+str(star_rating)] = temp.map(lambda x: int(x[star_rating-1]))

#### Sum star counts
df_book_info['Star_count_total'] = temp.map(lambda x: sum(map(int,x)))

#### Clean pages
print "Number of floats/int pages: ", df_book_info['Pages'].map(lambda x: isinstance(x, (int, float))).sum()
df_book_info['Pages'] = df_book_info['Pages'].map(lambda x: int((x.split())[0]) if not isinstance(x, (int, float)) else 0)


print df_book_info.applymap(lambda x: isinstance(x, (int, float))).all(0)


Is any star nan:  False
Number of non digit editions:  3
5
4
3
2
1
Number of floats/int pages:  0
Awards List         False
Book Name           False
Characters          False
Genres              False
Num_reviews          True
Other editions       True
Pages                True
Star votes          False
Stars                True
publish_date        False
Award Count          True
New Book Name       False
Num Characters       True
Star_count_5         True
Star_count_4         True
Star_count_3         True
Star_count_2         True
Star_count_1         True
Star_count_total     True
dtype: bool


In [4]:
genre_dict = [('Fiction', 1352), ('Fantasy', 630), ('Classics', 591), ('Mystery', 540), ('Young Adult', 420), ('Romance', 405), ('Literature', 396), ('Historical Fiction', 350), ('Thriller', 313), ('Childrens', 283), ('Science Fiction', 275), ('Novels', 227), ('Contemporary', 225), ('Adventure', 191), ('Crime', 188), ('Cultural', 187), ('Sequential Art', 180), ('Historical', 174), ('Humor', 169), ('Horror', 165), ('Paranormal', 157), ('Nonfiction', 146), ('War', 130), ('European Literature', 124), ('Biography', 112), ('Womens Fiction', 106), ('Suspense', 102), ('Chick Lit', 101), ('Adult', 91), ('Autobiography', 91), ('History', 89)]
# temp = df_book_info['Genres'].astype('str').map(lambda x: x[1:-1].split(',') if (x!='nan') else '')
# temp = temp.map(lambda x: [i.strip() for i in x])

# arr = temp.values

# for i in range(len(arr)):
#     for j in range(len(arr[i])):
#         try:
#             genre_dict[arr[i][j].strip()] +=1
#         except Exception:
#             genre_dict[arr[i][j].strip()] = 1
            
# genre_dict = sorted(genre_dict.items(), key=operator.itemgetter(1), reverse=True)
# print genre_dict[:31]


def check(x,genre_num):
    #print genre_dict
    if (genre_dict[genre_num][0] in x):
        return 1
    else:
        return 0

for genre_num in range(0,31):
    df_book_info['Genre_'+genre_dict[genre_num][0]] = temp.map(lambda x: check(x,genre_num))


print len(df_book_info)
print len(df_book_info.groupby('New Book Name', as_index=False).max())



122
122


In [4]:



df_book_info.to_csv('goodreads_data_modified_test.csv')
